In [1]:
import networkx as nx
import numpy as np
from numpy import linalg
from scipy.sparse import find, csr_matrix, csr_array
from matplotlib import pyplot, patches
import random
import itertools
import matplotlib.pyplot as plt
import time
import collections
from IPython.display import clear_output
import heapq

np.random.seed(42)
random.seed(10)

# Functions

In [2]:
#function to drow the adjacency matrix
def draw_adjacency_matrix(G, node_order=None, partitions=[], colors=[]):
    """
    - G is a netorkx graph
    - node_order (optional) is a list of nodes, where each node in G
          appears exactly once
    - partitions is a list of node lists, where each node in G appears
          in exactly one node list
    - colors is a list of strings indicating what color each
          partition should be
    If partitions is specified, the same number of colors needs to be
    specified.
    """
    adjacency_matrix = nx.to_numpy_array(G, dtype=bool, nodelist=node_order)

    #Plot adjacency matrix in toned-down black and white
    fig = pyplot.figure(figsize=(5, 5)) # in inches
    pyplot.imshow(adjacency_matrix,
                  cmap="Greys",
                  interpolation="none")

    # The rest is just if you have sorted nodes by a partition and want to
    # highlight the module boundaries
    assert len(partitions) == len(colors)
    ax = pyplot.gca()
    for partition, color in zip(partitions, colors):
        current_idx = 0
        for module in partition:
            ax.add_patch(patches.Rectangle((current_idx, current_idx),
                                          len(module), # Width
                                          len(module), # Height
                                          facecolor="none",
                                          edgecolor=color,
                                          linewidth="1"))
            current_idx += len(module)

In [3]:
def create_graph(size = [50,50], probs = [[0.9,0.1], [0.1,0.9]], weights = 0.1):
  temp = nx.stochastic_block_model(size, probs, seed=0)
  A = nx.adjacency_matrix(temp)
  W = A.multiply(weights).tocsr()
  G = nx.from_numpy_array(W)
  return G,W,A

In [4]:
def rand_bin_array(K, N, h):
  arr = np.zeros(N)
  arr[:K]  = h
  np.random.shuffle(arr)
  return arr

In [5]:
def history_to_np(history, max_calls):
  x = np.zeros(max_calls)
  t = 0
  temp = history[0][0]
  for i in range(max_calls):
    if history[t][1] == i+1:
      temp = history[t][0]
      if t != len(history)-1:
        t+=1
    x[i] = temp
  return x

# Models

In [6]:
def Influence_evaluation2(g, W, x0, params, max_t = 999):

  l0, h0, theta_l, theta_h, gamma, eps = params

  s = 0

  st = [0]

  x = [x0]

  alpha = 0.1

#   for u, v, d in g.edges(data=True):
#     alpha += d["weight"]

#   alpha = alpha/len(g.edges)

  t = 1

  while np.linalg.norm(x[-1]*((1-gamma)**t)) > eps and t <= max_t:

    l = ((theta_l*alpha)**t)*l0
    h = (theta_h*(theta_l**(t-1))*(alpha**t))*h0

    xt = GIP_function(np.sum(np.multiply(W,x[-1]), axis = 1), h, l)
    s += np.sum(xt)

    x.append(xt)

    st.append(s)
    t += 1

  return s, st, x

In [7]:
def Influence_evaluation(g, W, x0, params, max_t = 999):

  l0, h0, theta_l, theta_h, gamma, eps = params


  s = 0

  st = [0]

  x = [x0]

  alpha = 0.1

  # for u, v, d in g.edges(data=True):
  #   alpha += d["weight"]

  # alpha = alpha/len(g.edges)

  t = 1


  while np.linalg.norm(x[-1]*((1-gamma)**t)) > eps and t <= max_t:


    l = ((theta_l*alpha)**t)*l0
    h = (theta_h*(theta_l**(t-1))*(alpha**t))*h0

    xt = GIP_function(W.multiply(x[-1]).sum(axis = 1), h, l)
    s += np.sum(xt)

    x.append(xt)

    st.append(s)
    t += 1

  return s, st, x

In [8]:
def GIP_function(x,h,l):
  r = np.multiply(x >= l, x)
  return np.minimum(r, h)

# Solver

In [9]:
def brute_force_solver(g, W, params):
  N = len(g.nodes)
  X = itertools.combinations(range(N), K)
  s = 0
  x_res = np.zeros(N)
  for elem in X:
    x_temp = np.zeros(N)
    x_temp[list(elem)]  = h0
    s_temp = Influence_evaluation(g, W, x_temp, params)[0]
    if s_temp > s:
      x_res = x_temp
      s = s_temp
  return s, x_res

In [10]:
def RS_solver(g, W, params, K, max_calls):
  N = len(g.nodes)
  calls = 0
  s = 0
  history = []
  start = time.time()

  while calls < max_calls:
    x_temp = rand_bin_array(K, N, params[1])
    s_temp =  Influence_evaluation(g, W, x_temp, params)[0]
    calls += 1
    print("\r" + "Random search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

    if s_temp > s:
      s = s_temp
      X = x_temp
      history.append([s, calls])

  history.append([s, calls])

  print("\r" + "Random search... {}/{} Done!".format(calls, max_calls))
  print("s: {}".format(s))

  return s, X, history


In [11]:
def CDS_solver(g, W, x0, params, delta, xi, d, max_calls, buffer_dim):

  N = len(x0)
  K = np.count_nonzero(x0)
  X = [x0]
  s = [Influence_evaluation(g, W, x0, params)[0]]
  r = 0
  xi_t = xi

  stop = False
  buffer = collections.deque(maxlen = buffer_dim)
  calls = 1
  start = time.time()
  history = [[s[-1],calls]]

  print("\r" + "Custom direct search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

  while (stop == False) and (r<1000):
    idx = set(np.nonzero(X[-1])[0])
    neighbors = []
    idx_temp = set(range(N))-set(idx)

    for i in range(d//2):
      for elem1 in itertools.combinations(idx, len(idx)-1-i):
        for elem2 in itertools.combinations(idx_temp, i+1):
          neighbors.append(list(set(elem1) | set(elem2)))


    s_temp = s[-1]
    x_temp = X[-1]

    for elem in neighbors:
      x_elem = np.zeros(N)
      x_elem[list(elem)]  = h0
      if list(x_elem) in buffer:
        continue
      if calls >= max_calls:
        stop = True
        break

      s_elem = Influence_evaluation(g, W, x_elem, params)[0]
      buffer.append(list(x_elem))
      calls += 1
      print("\r" + "Custom direct search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

      if s_elem > s_temp:
        history.append([s_elem,calls])
        x_temp = x_elem
        s_temp = s_elem
        if s_temp > (1+xi_t)*s[-1]:
          break

    X.append(x_temp)
    s.append(s_temp)
    r += 1

    if s[-1] == s[-2]:
      X.pop()
      s.pop()
      stop = True
    elif s[-1] > (1+xi_t)*s[-2]:
      continue
    else:
      xi_t = xi_t * delta

  history.append([s[-1], calls])

  print("\r" + "Custom direct search... {}/{} Done!".format(calls, max_calls))
  print("s: {}".format(s[-1]))

  return s, X, history

In [12]:
def CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim):

  N = len(x0)
  K = np.count_nonzero(x0)
  X = [x0]
  s = [Influence_evaluation(g, W, x0, params)[0]]
  r = 0
  xi_t = xi

  stop = False
  buffer = collections.deque(maxlen = buffer_dim)
  calls = 1
  start = time.time()
  history = [[s[-1],time.time()-start, calls]]

  print("\r" + "Custom direct search... Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]), end = "")

  while (stop == False) and (r<1000):
    idx = set(np.nonzero(X[-1])[0])
    neighbors = []
    idx_temp = set(range(N))-set(idx)

    for i in range(d//2):
      for elem1 in itertools.combinations(idx, len(idx)-1-i):
        for elem2 in itertools.combinations(idx_temp, i+1):
          neighbors.append(list(set(elem1) | set(elem2)))


    s_temp = s[-1]
    x_temp = X[-1]

    for elem in neighbors:
      x_elem = np.zeros(N)
      x_elem[list(elem)]  = h0
      if list(x_elem) in buffer:
        continue
      if time.time()-start >= max_time:
        stop = True
        break

      s_elem = Influence_evaluation(g, W, x_elem, params)[0]
      buffer.append(list(x_elem))
      calls += 1
      print("\r" + "Custom direct search... Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]), end = "")

      if s_elem > s_temp:
        history.append([s_elem,time.time()-start, calls])
        x_temp = x_elem
        s_temp = s_elem
        if s_temp > (1+xi_t)*s[-1]:
          break

    X.append(x_temp)
    s.append(s_temp)
    r += 1

    if s[-1] == s[-2]:
      X.pop()
      s.pop()
      stop = True
    elif s[-1] > (1+xi_t)*s[-2]:
      continue
    else:
      xi_t = xi_t * delta

  history.append([s[-1], time.time()-start, calls])   

  print("\r" + "Custom direct search... Done! Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]))

  return s, X, history

In [13]:
def NS_solver(g, W, x0, params, delta, xi, d, max_calls, buffer_dim):

  N = len(x0)
  K = np.count_nonzero(x0)
  X = [x0]
  s = [Influence_evaluation(g, W, x0, params)[0]]
  r = 0
  xi_t = xi

  stop = False
  buffer = collections.deque(maxlen = buffer_dim)
  calls = 1
  start = time.time()
  history = [[s[-1],calls]]

  print("\r" + "Neighbors search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

  while (stop == False) and (r<1000):
    idx = set(np.nonzero(X[-1])[0])
    neighbors = []

    for elem1 in idx:
      for elem2 in set(g.neighbors(elem1))-idx:
        temp = idx.copy()
        temp.add(elem2)
        temp.remove(elem1)
        neighbors.append(temp)

    s_temp = s[-1]
    x_temp = X[-1]

    for elem in neighbors:
      x_elem = np.zeros(N)
      x_elem[list(elem)]  = h0
      if list(x_elem) in buffer:
        continue
      if calls >= max_calls:
        stop = True
        break

      s_elem = Influence_evaluation(g, W, x_elem, params)[0]
      buffer.append(list(x_elem))
      calls +=1
      print("\r" + "Neighbors search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

      if s_elem > s_temp:
        history.append([s_elem,calls])
        x_temp = x_elem
        s_temp = s_elem
        if s_temp > (1+xi_t)*s[-1]:
          break

    X.append(x_temp)
    s.append(s_temp)
    r += 1

    if s[-1] > (1+xi_t)*s[-2]:
      continue

    elif s[-1] > s[-2]:
      xi_t = xi_t * delta

    else:
      X.pop()
      s.pop()
      idx = set(np.nonzero(X[-1])[0])
      neighbors = []
      idx_temp = set(range(N))-set(idx)

      for i in range(d//2):
        for elem1 in itertools.combinations(idx, len(idx)-1-i):
          for elem2 in itertools.combinations(idx_temp, i+1):
            neighbors.append(list(set(elem1) | set(elem2)))

      s_temp = s[-1]
      x_temp = X[-1]

      for elem in neighbors:
        x_elem = np.zeros(N)
        x_elem[list(elem)]  = h0
        if list(x_elem) in buffer:
          continue
        if calls >= max_calls:
          stop = True
          break

        s_elem = Influence_evaluation(g, W, x_elem, params)[0]
        buffer.append(list(x_elem))
        calls += 1
        print("\r" + "Neighbors search... {}/{}. ETA: {} s.".format(calls, max_calls, round((time.time()-start)*((max_calls-calls)/calls))), end = "")

        if s_elem > s_temp:
          history.append([s_elem,calls])
          x_temp = x_elem
          s_temp = s_elem
          if s_temp > (1+xi_t)*s[-1]:
            break

      X.append(x_temp)
      s.append(s_temp)
      r += 1

      if s[-1] == s[-2]:
        X.pop()
        s.pop()
        stop = True

      elif s[-1] > (1+xi_t)*s[-2]:
        continue

      else:
        xi_t = xi_t * delta

  history.append([s[-1], calls])

  print("\r" + "Neighbors search... {}/{} Done!".format(calls, max_calls))
  print("s: {}".format(s[-1]))

  return s, X, history

In [14]:
def NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim):

  N = len(x0)
  K = np.count_nonzero(x0)
  X = [x0]
  s = [Influence_evaluation(g, W, x0, params)[0]]
  r = 0
  xi_t = xi

  stop = False
  buffer = collections.deque(maxlen = buffer_dim)
  calls = 1
  start = time.time()
  history = [[s[-1],time.time()-start, calls]]

  print("\r" + "Neighbors search... Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]), end = "")

  while (stop == False) and (r<1000):
    idx = set(np.nonzero(X[-1])[0])
    neighbors = []

    for elem1 in idx:
      for elem2 in set(g.neighbors(elem1))-idx:
        temp = idx.copy()
        temp.add(elem2)
        temp.remove(elem1)
        neighbors.append(temp)

    s_temp = s[-1]
    x_temp = X[-1]

    for elem in neighbors:
      x_elem = np.zeros(N)
      x_elem[list(elem)]  = h0
      if list(x_elem) in buffer:
        continue
      if time.time()-start >= max_time:
        stop = True
        break

      s_elem = Influence_evaluation(g, W, x_elem, params)[0]
      buffer.append(list(x_elem))
      calls +=1
      print("\r" + "Neighbors search... Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]), end = "")

      if s_elem > s_temp:
        history.append([s_elem, time.time()-start, calls])
        x_temp = x_elem
        s_temp = s_elem
        if s_temp > (1+xi_t)*s[-1]:
          break

    X.append(x_temp)
    s.append(s_temp)
    r += 1

    if s[-1] > (1+xi_t)*s[-2]:
      continue

    elif s[-1] > s[-2]:
      xi_t = xi_t * delta

    else:
      X.pop()
      s.pop()
      idx = set(np.nonzero(X[-1])[0])
      neighbors = []
      idx_temp = set(range(N))-set(idx)

      for i in range(d//2):
        for elem1 in itertools.combinations(idx, len(idx)-1-i):
          for elem2 in itertools.combinations(idx_temp, i+1):
            neighbors.append(list(set(elem1) | set(elem2)))

      s_temp = s[-1]
      x_temp = X[-1]

      for elem in neighbors:
        x_elem = np.zeros(N)
        x_elem[list(elem)]  = h0
        if list(x_elem) in buffer:
          continue
        if time.time()-start >= max_time:
          stop = True
          break

        s_elem = Influence_evaluation(g, W, x_elem, params)[0]
        buffer.append(list(x_elem))
        calls += 1
        print("\r" + "Neighbors search... Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]), end = "")
        if s_elem > s_temp:
          history.append([s_elem, time.time() - start, calls])
          x_temp = x_elem
          s_temp = s_elem
          if s_temp > (1+xi_t)*s[-1]:
            break

      X.append(x_temp)
      s.append(s_temp)
      r += 1

      if s[-1] == s[-2]:
        X.pop()
        s.pop()
        stop = True

      elif s[-1] > (1+xi_t)*s[-2]:
        continue

      else:
        xi_t = xi_t * delta

  history.append([s[-1], time.time()-start, calls])

  print("\r" + "Neighbors search... Done! Calls:{}. Time: {}/{} s. Influence spread: {}.".format(calls, round(time.time()-start), max_time, s[-1]))

  return s, X, history

In [15]:
def greedy(g, W, params, K):
  N = len(g.nodes)
  temp = [0.0 for i in range(N)]
  s_out = 0.0
  for i in range(K):
    idx = 0
    value = 0.0
    for k in range(N):
      if temp[k] == 0:
        temp2 = temp.copy()
        temp2[k] = params[1]
        s = Influence_evaluation(g, W, np.array(temp2), params)[0]
        if s > value:
          value = s
          idx = k
          s_out = s

    temp[idx] = params[1]

  return s_out, temp


In [16]:
def SingleDiscount(g,W, params, K):
  W1 = W.toarray()
  idx = []
  ddv = np.sum(W1 > 0, axis = 0)
  tv = np.zeros(len(g.nodes))
  for i in range(K):
    u = np.argmax(ddv)
    idx.append(u)
    v = list(np.nonzero(W1[u][0]))
    tv[v] += 1
    ddv[v] = ddv[v] - tv[v]
    ddv[u] = -1
    W1[u,:] = 0
    W1[:,u] = 0

  X = np.zeros(len(g.nodes))
  X[idx] = params[1]
  S = Influence_evaluation(g, W, X, params)[0]

  return S, X

In [17]:
def RIS(g,W, params, K, R):
  H = nx.Graph()
  H.add_nodes_from(g.nodes)
  for i in range(R):
    idx = random.randint(0, len(g.nodes)-1)
    temp = np.zeros(len(g.nodes))
    temp[idx] = params[1]
    x = Influence_evaluation(g, W, temp, params)[2][1:]
    edges = []
    for elem in x:
      edges += list(np.nonzero(elem)[0])
    edges = list(set(edges)-set([idx]))
    edges = [(idx, elem) for elem in edges]
    H.add_edges_from(edges)
  H = H.to_undirected()
  W1 = nx.adjacency_matrix(H).toarray()

  t = []
  for i in range(K):
    W_temp = np.sum(W1, axis = 0)
    v = np.argmax(W_temp)
    t.append(v)
    W1[v,:] = 0
    W1[:,v] = 0

  X = np.zeros(len(g.nodes))
  X[t] = params[1]
  S = Influence_evaluation(g, W, X, params)[0]

  return S, X

# Experiments

In [18]:
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv big\\CA-HepPh.txt", create_using=nx.DiGraph(), nodetype = int)
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv small\\com-dblp.ungraph.txt", create_using=nx.DiGraph(), nodetype = int)
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Networks\\Facebook\\facebook_combined.txt", create_using=nx.DiGraph(), nodetype = int)
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv Astro\\Arxive_astro.txt", create_using=nx.DiGraph(), nodetype = int)
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv GR-QC\\CA-GrQc.txt", create_using=nx.DiGraph(), nodetype = int)
# g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\Email-Enron.txt", create_using=nx.DiGraph(), nodetype = int)
g = nx.read_edgelist("C:\\Users\\Matteo\\Desktop\\lastfm_asia.txt", create_using=nx.DiGraph(), nodetype = int)

In [19]:
# with open('C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv small\\community_1.txt') as f:
#    for l in f:
#        c1 = l.strip().split("\t")
# with open('C:\\Users\\Matteo\\Desktop\\Networks\\Arxiv small\\community_2.txt') as f:
#    for l in f:
#        c2 = l.strip().split("\t")
# c1 = [int(elem) for elem in c1]
# c2 = [int(elem) for elem in c2]
# c = list(set(c1+c2))

# g = g.subgraph(c)

In [20]:
alpha = 0.1
g = g.to_undirected()
A = nx.adjacency_matrix(g)
W = A.multiply(alpha)
W1 = W.toarray()
g = nx.from_numpy_array(W1)

In [21]:
print(len(g.nodes),len(g.edges))

7624 27806


In [22]:
# l0 = 1
# h0 = 1
# theta_l = 2
# theta_h = 50
# gamma = 0
# eps = 0.1

# params = l0, h0, theta_l, theta_h, gamma, eps

# delta = 0.5
# xi = 0.1
# d = 2

# buffer_dim = 500

# R = len(g.nodes)

# katz = []
# ns_katz = []
# ns_disc = []
# cds_katz = []
# cds_disc = []
# disc = []
# greed = []
# ris = []

# K0 = 5
# K1 = 5

# for K in range(K0, K1 + 1):

#   max_time = K*10
    
#   print(K)

#   start = time.time()
    
#   x0 = np.zeros(len(g.nodes))
#   x0[list(np.argsort(np.array(list(nx.katz_centrality_numpy(g).values())))[-K:])] = h0
    
#   katz.append((Influence_evaluation(g, W, x0, params)[0],time.time()-start))
#   print('katz:',katz[-1][0])

#   start = time.time()
#   s, X, history = NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#   ns_katz.append((s[-1], time.time()-start, history))

#   start = time.time()
#   s, X, history = CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#   cds_katz.append((s[-1], time.time()-start, history))

#   start = time.time()
#   s, x0 = SingleDiscount(g, W, params, K)
#   disc.append((s,time.time()-start))
#   print('disc:',disc[-1][0])

#   start = time.time()
#   ris.append((RIS(g, W, params, K, R)[0],time.time()-start))
#   print('ris:',ris[-1][0])

#   start = time.time()
#   greed.append((greedy(g, W, params, K)[0],time.time()-start))
#   print('greed:',greed[-1][0])

#   start = time.time()
#   s, X, history = NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#   ns_disc.append((s[-1], time.time()-start, history))

#   start = time.time()
#   s, X, history = CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#   cds_disc.append((s[-1], time.time()-start, history))

In [ ]:
l0 = 1
h0 = 1
theta_l = 2
theta_h = 50
gamma = 0
eps = 0.1

params = l0, h0, theta_l, theta_h, gamma, eps

delta = 0.5
xi = 0.1
d = 2

buffer_dim = 500

R = len(g.nodes)

disc = []
katz = []
ns_disc = []
cds_disc = []
greed = []
ris = []


for K in [5,10,15,20]:
  ns = []
  cds = []
    
  max_time = K*200
    
  print(K)

  start = time.time()  
  x0 = np.zeros(len(g.nodes))
  x0[list(np.argsort(np.array(list(nx.katz_centrality_numpy(g).values())))[-K:])] = h0  
  katz.append((Influence_evaluation(g, W, x0, params)[0],time.time()-start))
  print('katz:',katz[-1][0])

  start = time.time()
  s, x0 = SingleDiscount(g, W, params, K)
  disc.append((s,time.time()-start))
  print('disc:',disc[-1][0])

  start = time.time()
  ris.append((RIS(g, W, params, K, R)[0],time.time()-start))
  print('ris:',ris[-1][0])

  start = time.time()
  greed.append((greedy(g, W, params, K)[0],time.time()-start))
  print('greed:',greed[-1][0],greed[-1][1])

  start = time.time()
  s, X, history = NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
  ns_disc.append((s[-1], time.time()-start, history))
  np.savetxt("ns_disc_" + str(K) + ".csv", ns_disc[-1][2])

  start = time.time()
  s, X, history = CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
  cds_disc.append((s[-1], time.time()-start, history))
  np.savetxt("cds_disc_" + str(K) + ".csv", cds_disc[-1][2])
    
  top_nc = list(np.argsort(np.sum(W1 > 0, axis = 0))[-4*K:])
    
  for i1 in range(10):
    x0 = np.zeros(len(g.nodes))
    x0[list(random.sample(top_nc, K))] = params[1]
    print('rand_nc: ' + str(Influence_evaluation(g, W, x0, params)[0]))

    s, X, history = NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
    t = []
    value = 0
    for j in range(max_time+1):
      for elem in history:
        if elem[1]> j:
          continue
        if elem[0] > value:
          value = elem[0]
      t.append(value)
    ns.append(t)

    s, X, history = CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
    t = []
    value = 0
    for j in range(max_time+1):
      for elem in history:
        if elem[1]> j:
          continue
        if elem[0] > value:
          value = elem[0]
      t.append(value)
    cds.append(t)
  np.savetxt("ns_{}.csv".format(K), ns)
  np.savetxt("cds_{}.csv".format(K), cds)

5
katz: 126.5441928
disc: 187.99936920000005
ris: 0.0
greed: 178.69744130000004 111.77482438087463
Neighbors search... Done! Calls:23538. Time: 1000/1000 s. Influence spread: 406.41405530000014.
Custom direct search... Done! Calls:20156. Time: 1000/1000 s. Influence spread: 407.79535500000014.
rand_nc: 359.87813410000007
Neighbors search... Done! Calls:22018. Time: 1000/1000 s. Influence spread: 487.0611756000001.
Custom direct search... Done! Calls:19968. Time: 1000/1000 s. Influence spread: 449.75977050000006.
rand_nc: 224.06962400000006
Neighbors search... Done! Calls:22465. Time: 1000/1000 s. Influence spread: 454.4212170000002.
Custom direct search... Done! Calls:19907. Time: 1000/1000 s. Influence spread: 275.4318049000001.
rand_nc: 194.73144040000005
Neighbors search... Done! Calls:21667. Time: 1000/1000 s. Influence spread: 487.0611756000001.
Custom direct search... Done! Calls:19731. Time: 1000/1000 s. Influence spread: 362.2997887000001.
rand_nc: 299.82594220000016
Neighbors 

In [ ]:
# np.savetxt("ns_{}.csv".format(K), ns)
# np.savetxt("cds_{}.csv".format(K), cds)

In [ ]:
# names = ['ns_disc','ns_katz','cds_disc','cds_katz','nmns_disc','nmns_katz','katz','disc','greed','ris']
names = ['greed','disc','katz','ris']
times = [[elem[1] for elem in eval(name)] for name in names]
scores = [[elem[0] for elem in eval(name)] for name in names]


In [ ]:
# l0 = 1
# h0 = 1
# theta_l = 2
# theta_h = 50
# gamma = 0
# eps = 0.1

# params = l0, h0, theta_l, theta_h, gamma, eps

# delta = 0.5
# xi = 0.1
# d = 2

# buffer_dim = 500

# R = len(g.nodes)

# disc = []
# ns = []
# cds = []

# K0 = 10
# K1 = 10

# for K in range(K0, K1 + 1):

#   max_time = K*100
    
#   print(K)
    
#   start = time.time()
#   s, x0 = SingleDiscount(g, W, params, K)
#   disc.append((s,time.time()-start))
#   print('disc:',disc[-1][0])
      
#   top_nc = list(np.argsort(np.sum(W1 > 0, axis = 0))[-4*K:])
#   for i in range(10):
#     x0 = np.zeros(len(g.nodes))
#     x0[list(random.sample(top_nc, K))] = params[1]
#     print('rand_nc: ' + str(Influence_evaluation(g, W, x0, params)[0]))

#     s, X, history = NS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#     t = []
#     value = 0
#     for i in range(max_time+1):
#       for elem in history:
#         if elem[1]> i:
#           continue
#         if elem[0] > value:
#           value = elem[0]
#       t.append(value)
#     ns.append(t)

#     s, X, history = CDS_solver(g, W, x0, params, delta, xi, d, max_time, buffer_dim)
#     t = []
#     value = 0
#     for i in range(max_time+1):
#       for elem in history:
#         if elem[1]> i:
#           continue
#         if elem[0] > value:
#           value = elem[0]
#       t.append(value)
#     cds.append(t)

In [ ]:
np.savetxt("times.csv", times)
np.savetxt("scores.csv", scores)

In [ ]:
f
ns_median = []
ns_mean = []
ns_std = []
cds_median = []
cds_mean = []
cds_std = []
ns_np = np.array(ns)
cds_np = np.array(cds)
for i in range(max_time+1):
  ns_median.append(np.median(ns_np[:,i]))
  ns_mean.append(np.mean(ns_np[:,i]))
  ns_std.append(np.std(ns_np[:,i]))
  cds_median.append(np.median(cds_np[:,i]))
  cds_mean.append(np.mean(cds_np[:,i]))
  cds_std.append(np.std(cds_np[:,i]))

In [ ]:
plt.plot(ns_mean)
plt.plot(cds_mean)
plt.show()

In [ ]:
names = ['ns_disc','ns_katz','cds_disc','cds_katz','nmns_disc','nmns_katz','katz','disc','greed','ris']
names = ['ns_disc','ns_katz','cds_disc','cds_katz','katz','disc','greed','ris']
for name in names:
    print(name, eval(name))

In [ ]:
names = ['ns_disc','ns_katz','cds_disc','cds_katz']
for name in names:
  for i,elem in enumerate(eval(name)):
    np.savetxt(name + "_" + str(K0+i) + ".csv", elem[2])